-without denoising of the training and testing data

-detection of anomalies on the test dataset with added anomalies

In [ ]:
import pandas as pd
import datetime
import numpy as np
import time

In [ ]:
# Load your dataset
file_path = 'top10_hourly.csv'
data = pd.read_csv(file_path)

In [ ]:
six_grids = (5059, 5159, 5262, 5061, 5259, 6064)
mask = data['gridID'].isin(six_grids)
data = data[mask]

In [ ]:
# Convert startTime to datetime
data['startTime'] = pd.to_datetime(data['startTime'])

In [ ]:
# Filter the data to keep only records on or before 2013-12-22
filtered_data = data[data['startTime'] <= '2013-12-22']

In [ ]:
# Create anomaly marker columns for relevant features
filtered_data['anomaly'] = 0

<ipython-input-6-a54f7f6a1c87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['anomaly'] = 0


In [ ]:
# Splitting the dataset to train (80%) and test (20%)
# Sorting the data by 'startTime' to ensure the chronological order is maintained.                         ####

sorted_data = filtered_data.sort_values(by='startTime')

# Now, we calculate the index for the 80/20 split
split_index = int(len(sorted_data) * 0.8)

# Split the data into train and test sets
train_data = sorted_data.iloc[:split_index]
test_data = sorted_data.iloc[split_index:]

In [ ]:
for grid in test_data['gridID'].unique():
    anomaly_mask_1 = (test_data['gridID'] == grid) & (test_data['startTime'].dt.date == datetime.datetime(2013, 12, 18).date()) \
                 & (test_data['startTime'].dt.hour >= 10) & (test_data['startTime'].dt.hour <= 20)
    test_data.loc[anomaly_mask_1, 'smsIn'] *= 0.1  # Drop in SMS In
    test_data.loc[anomaly_mask_1, 'anomaly'] = 2  # Mark as smsIn anomaly

    anomaly_mask_2 = (test_data['gridID'] == grid) & (test_data['startTime'].dt.date == datetime.datetime(2013, 12, 16).date()) \
                 & (test_data['startTime'].dt.hour >= 10) & (test_data['startTime'].dt.hour <= 20)
    test_data.loc[anomaly_mask_2, 'callOut'] *= 0.1  # Drop in Call Out
    test_data.loc[anomaly_mask_2, 'anomaly'] = 3  # Mark as callOut anomaly


anomaly_mask = (test_data['gridID'] == 5059) & (test_data['startTime'].dt.date == datetime.datetime(2013, 12, 14).date()) \
                 & (test_data['startTime'].dt.hour >= 10) & (test_data['startTime'].dt.hour <= 20)
test_data.loc[anomaly_mask, 'internet'] *= 2.5  # Spike in internet usage
test_data.loc[anomaly_mask, 'anomaly'] = 1  # Mark as internet anomaly

In [ ]:
test_data_abnormal = test_data[test_data['anomaly'] !=0]


In [ ]:
outliers_1 = pd.DataFrame()
start_time = time.time()  # Start timer
for grid in test_data.gridID.unique():
    data_grid = test_data[test_data['gridID'] == grid]

    for column in ['internet', 'smsIn', 'smsOut', 'callIn', 'callOut']:
        values = data_grid[column]

        # Calculate IQR
        Q1 = np.percentile(values, 25)
        Q3 = np.percentile(values, 75)
        IQR = Q3 - Q1

        # Define outlier boundaries
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Identify outliers
        outlier_indices = data_grid[(data_grid[column] < lower_bound) | (data_grid[column] > upper_bound)].index
        # Replace outliers with the column median
        #data_1.loc[outlier_indices, column] = data_grid[column].median()

        #Replace outliers with the column mean
        #data_1.loc[outlier_indices, column] = data_grid[column].mean()

        #Replace outliers with the rolling mean
        test_data.loc[outlier_indices, column] = data_grid[column].rolling(window = 10, center = True, min_periods = 5).mean()
        outliers_1 = pd.concat([outliers_1, data_grid.loc[outlier_indices]])
end_time = time.time()  # End timer
execution_time = end_time - start_time  # Compute execution time

print(f"Execution Time: {execution_time:.4f} seconds")  # Print execution time

Execution Time: 0.1110 seconds


In [ ]:
outliers_1.shape

(10, 8)

In [ ]:
test_data_abnormal.head()

,gridID,startTime,smsIn,smsOut,callIn,callOut,internet,anomaly
4018,5059,2013-12-14 10:00:00,876.768421,385.107740,548.545813,671.959604,18605.611282,1
4019,5059,2013-12-14 11:00:00,1479.136379,556.760997,941.248122,1068.046517,28787.036181,1
4020,5059,2013-12-14 12:00:00,1829.513298,581.343243,1050.937450,1413.252409,38738.167569,1
4021,5059,2013-12-14 13:00:00,1922.505204,896.034323,1102.463478,1290.052171,44261.699773,1
4022,5059,2013-12-14 14:00:00,1789.207042,742.470167,928.660964,1149.592928,48423.608645,1


In [ ]:
outliers_1.head()

,gridID,startTime,smsIn,smsOut,callIn,callOut,internet,anomaly
4020,5059,2013-12-14 12:00:00,1829.513298,581.343243,1050.937450,1413.252409,38738.167569,1
4021,5059,2013-12-14 13:00:00,1922.505204,896.034323,1102.463478,1290.052171,44261.699773,1
4022,5059,2013-12-14 14:00:00,1789.207042,742.470167,928.660964,1149.592928,48423.608645,1
4023,5059,2013-12-14 15:00:00,1995.964117,827.534748,1151.933630,1428.129267,46455.539809,1
4024,5059,2013-12-14 16:00:00,2574.084658,1003.645021,1292.320932,1728.981653,48361.531999,1


In [ ]:
# Merge the two dataframes on matching GridID and Timestamp/startTime
merged_df = pd.merge(test_data_abnormal, outliers_1, left_on=['startTime', 'gridID'], right_on=['startTime', 'gridID'], how='inner')

In [ ]:
merged_df

,gridID,startTime,smsIn_x,smsOut_x,callIn_x,callOut_x,internet_x,anomaly_x,smsIn_y,smsOut_y,callIn_y,callOut_y,internet_y,anomaly_y
0,5059,2013-12-14 12:00:00,1829.513298,581.343243,1050.937450,1413.252409,38738.167569,1,1829.513298,581.343243,1050.937450,1413.252409,38738.167569,1
1,5059,2013-12-14 13:00:00,1922.505204,896.034323,1102.463478,1290.052171,44261.699773,1,1922.505204,896.034323,1102.463478,1290.052171,44261.699773,1
2,5059,2013-12-14 14:00:00,1789.207042,742.470167,928.660964,1149.592928,48423.608645,1,1789.207042,742.470167,928.660964,1149.592928,48423.608645,1
3,5059,2013-12-14 15:00:00,1995.964117,827.534748,1151.933630,1428.129267,46455.539809,1,1995.964117,827.534748,1151.933630,1428.129267,46455.539809,1
4,5059,2013-12-14 16:00:00,2574.084658,1003.645021,1292.320932,1728.981653,48361.531999,1,2574.084658,1003.645021,1292.320932,1728.981653,48361.531999,1
5,5059,2013-12-14 17:00:00,2457.668355,758.452259,1245.438764,1740.500048,43081.053201,1,2457.668355,758.452259,1245.438764,1740.500048,43081.053201,1
6,5059,2013-12-14 18:00:00,2116.968330,733.389263,1194.090082,1417.360181,42876.388437,1,2116.968330,733.389263,1194.090082,1417.360181,42876.388437,1
7,5059,2013-12-14 19:00:00,1425.306129,550.077523,849.887673,1045.667494,36597.148693,1,1425.306129,550.077523,849.887673,1045.667494,36597.148693,1
8,5259,2013-12-18 12:00:00,178.120013,2015.897308,1600.286521,1464.651851,19721.997519,2,178.120013,2015.897308,1600.286521,1464.651851,19721.997519,2
9,5259,2013-12-18 13:00:00,149.039347,1565.606142,1170.394887,1268.378868,20163.992547,2,149.039347,1565.606142,1170.394887,1268.378868,20163.992547,2


In [ ]:
merged_df.shape

(10, 14)

In [ ]:
print(f"{merged_df.shape[0]}/{test_data_abnormal.shape[0]}")

10/143
